In [48]:
import pandas as pd

# Replace 'file_path.csv' with the actual path to your CSV file
posts_df = pd.read_csv('./csv/Posts.csv')
Tags = pd.read_csv('./csv/Tags.csv')

answers_df = posts_df[posts_df['PostTypeId'] == 2][['Id', 'OwnerUserId', 'ParentId']]
answers_df['ParentId'] = answers_df['ParentId'].astype(int)
print(len(answers_df))

# Step 4: Extract the questions with their tags (where 'PostTypeId == 1')
questions_df = posts_df[posts_df['PostTypeId'] == 1][['Id', 'Tags']]

# Step 5: Ensure the question Ids are also of type int64
questions_df['Id'] = questions_df['Id'].astype(int)
print(len(questions_df))


# Step 3: Merge answers with the corresponding tags from the question (use ParentId to match question Id)
merged_df = pd.merge(answers_df, questions_df, left_on='ParentId', right_on='Id', suffixes=('_answer', '_question'))

# Step 4: Select relevant columns
filtered_answers_df = merged_df[['Id_answer', 'OwnerUserId', 'Tags']]  # 'Tags' here are from the question
answerer_counts = filtered_answers_df.groupby('OwnerUserId').size()
qualified_answerers = answerer_counts[answerer_counts >= 20].index
print("Qualified Answerers:", len(qualified_answerers))
filtered_answers = filtered_answers_df[filtered_answers_df['OwnerUserId'].isin(qualified_answerers)]
# print(len(filtered_answers))

qualified_tags = Tags[Tags['Count'] >= 20]['Id']
print("Qualified Tags:", len(qualified_tags))

tag_dict = Tags.set_index('Id')['TagName'].to_dict()




178628
63423
Qualified Answerers: 1163
Qualified Tags: 974
Tag Dictionary: {1: 'comments', 3: 'code-smell', 4: 'programming-languages', 5: 'usage', 7: 'business', 8: 'project-management', 9: 'development-process', 11: 'performance', 12: 'time-management', 13: 'dynamic-typing', 14: 'static-typing', 16: 'team', 23: 'waterfall', 28: 'functional-programming', 36: 'web-applications', 38: 'hiring', 40: 'resources', 41: 'math', 45: 'billing', 46: 'freelancing', 47: 'pricing', 49: 'twitter', 50: 'software', 51: 'technology', 52: 'books', 53: 'actor-model', 54: 'concurrency', 55: 'code-quality', 56: 'test-coverage', 57: 'open-source', 58: 'history', 61: 'linux', 62: 'kernel', 63: 'resume', 65: 'agile', 67: 'project-hosting', 71: 'productivity', 76: 'java', 78: 'python', 82: 'applications', 83: 'tools', 87: 'telecommuting', 89: 'self-improvement', 91: 'websites', 92: 'organization', 94: 'technique', 96: 'social', 102: 'language-design', 104: 'syntax', 105: 'learning', 109: 'regular-expressions',